In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from PIL import *
import os
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
from sklearn.metrics import *

C:\Users\amith\AppData\Local\Temp\ipykernel_18536\1338435797.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
tumour_dataset_path = os.getcwd() + "\data"

image_files = []
labels = []
ctr = 0

lab = []
for i in os.listdir(tumour_dataset_path):
    lab.append(i)
lab = sorted(lab)
    
for i in lab:
    k = tumour_dataset_path + "\\" + i + "\\"
    for j in os.listdir(k):
        image_files.append(k + j)
        labels.append(ctr)
    ctr += 1
image_files = np.array(image_files)
labels = np.array(labels)
labels_1 = labels.copy()

In [3]:
images = []
images_1 = []

for i in image_files:
    image_original = Image.open(i)
    image_grayscale = image_original.convert("L")
    image_resized = image_grayscale.resize((64,64))
    image_array = np.array(image_resized) / 255.0
    image_array = image_array.reshape((64,64,1))
    images.append(image_array)

    image_resized_1 = image_grayscale.resize((256,256))
    image_array_1 = np.array(image_resized_1) / 255.0
    image_array_1 = image_array_1.reshape((256,256,1))
    images_1.append(image_array_1)

    
images = np.array(images)
images_1 = np.array(images_1)

In [4]:
images[0].shape

(64, 64, 1)

In [5]:
images_1[0].shape

(256, 256, 1)

In [6]:
image_positions = list(range(len(images)))
np.random.shuffle(image_positions)
image_labels_shuffled = labels[image_positions]
image_inputs_shuffled = images[image_positions]

image_positions_1 = list(range(len(images_1)))
np.random.shuffle(image_positions_1)
image_labels_shuffled_1 = labels_1[image_positions_1]
image_inputs_shuffled_1 = images_1[image_positions_1]

In [7]:
number_of_images = len(image_inputs_shuffled)

div_ratio = 0.25

test_size = round(number_of_images * div_ratio)
train_val_size = number_of_images - test_size
val_test_threshold = train_val_size

val_size = round(train_val_size * div_ratio)
train_size = train_val_size - val_size
train_val_threshold = train_size


number_of_images_1 = len(image_inputs_shuffled_1)

div_ratio_1 = 0.2

test_size_1 = round(number_of_images_1 * div_ratio_1)
train_val_size_1 = number_of_images_1 - test_size_1
val_test_threshold_1 = train_val_size_1

val_size_1 = round(train_val_size_1 * div_ratio_1)
train_size_1 = train_val_size_1 - val_size_1
train_val_threshold_1 = train_size_1

In [8]:
print(val_test_threshold)
print(train_val_threshold)

194
146


In [9]:
print(val_test_threshold_1)
print(train_val_threshold_1)

207
166


In [10]:
X_train = image_inputs_shuffled[:train_val_threshold]
y_train = image_labels_shuffled[:train_val_threshold]

X_val = image_inputs_shuffled[train_val_threshold:val_test_threshold]
y_val = image_labels_shuffled[train_val_threshold:val_test_threshold]

X_test = image_inputs_shuffled[val_test_threshold:]
y_test = image_labels_shuffled[val_test_threshold:]


X_train_1 = image_inputs_shuffled_1[:train_val_threshold_1]
y_train_1 = image_labels_shuffled_1[:train_val_threshold_1]

X_val_1 = image_inputs_shuffled_1[train_val_threshold_1:val_test_threshold_1]
y_val_1 = image_labels_shuffled_1[train_val_threshold_1:val_test_threshold_1]

X_test_1 = image_inputs_shuffled_1[val_test_threshold_1:]
y_test_1 = image_labels_shuffled_1[val_test_threshold_1:]

In [11]:
print(X_train.shape[0],X_val.shape[0],X_test.shape[0])
print(X_train_1.shape[0],X_val_1.shape[0],X_test_1.shape[0])

146 48 65
166 41 52


In [12]:
model = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation="relu", input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=64, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")  

])

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.binary_crossentropy, metrics=tf.keras.metrics.binary_accuracy)
model.fit(X_train,y_train,epochs=10,shuffle=True,validation_data=(X_val,y_val),verbose=1,callbacks=tf.keras.callbacks.EarlyStopping(patience=4))

Epoch 1/10


5/5 [==============================] - 3s 172ms/step - loss: 0.6957 - binary_accuracy: 0.5274 - val_loss: 0.6937 - val_binary_accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 0s 47ms/step - loss: 0.6811 - binary_accuracy: 0.6027 - val_loss: 0.7051 - val_binary_accuracy: 0.5000
Epoch 3/10
5/5 [==============================] - 0s 49ms/step - loss: 0.6704 - binary_accuracy: 0.6027 - val_loss: 0.7412 - val_binary_accuracy: 0.5000
Epoch 4/10
5/5 [==============================] - 0s 45ms/step - loss: 0.6775 - binary_accuracy: 0.6027 - val_loss: 0.7136 - val_binary_accuracy: 0.5000
Epoch 5/10
5/5 [==============================] - 0s 46ms/step - loss: 0.6717 - binary_accuracy: 0.6027 - val_loss: 0.7136 - val_binary_accuracy: 0.5000


In [14]:
y_pred = model.predict(X_test,verbose=0).round()
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", round(accuracy,2))

Accuracy: 0.63


In [15]:
pd.

SyntaxError: invalid syntax (3133375982.py, line 1)

In [16]:
test_path = os.getcwd() + "\\test\\"
test_images = []
test_images_1 = []
test_labels = []
test_labels_1 = []
gg = 00
oo = []
for i in os.listdir(test_path):
    image_original = Image.open(test_path + i)
    image_grayscale = image_original.convert("L")
    
    image_resized = image_grayscale.resize((64,64))
    image_array = np.array(image_resized) / 255.0
    image_array = image_array.reshape((64,64,1))
    test_images.append(image_array)


    image_resized_1 = image_grayscale.resize((256,256))
    image_array_1 = np.array(image_resized_1) / 255.0
    image_array_1 = image_array_1.reshape((256,256,1))
    test_images_1.append(image_array_1)

test_images = np.array(test_images)
test_images_1 = np.array(test_images_1)

In [17]:
test_images[0].shape

(64, 64, 1)

In [18]:
test_images_1[0].shape

(256, 256, 1)

In [19]:
test_labels = model.predict(test_images,verbose=0).round()

In [20]:
print(test_labels.reshape(1,-1)[0].tolist())

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [21]:
model_1 = tf.keras.models.Sequential(layers=[

    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),


    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")

])

In [22]:
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.binary_crossentropy, metrics=tf.keras.metrics.binary_accuracy)
model_1.fit(X_train_1,y_train_1,epochs=75,shuffle=True,validation_data=(X_val_1,y_val_1),verbose=1,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/75
6/6 [==============================] - 16s 2s/step - loss: 0.6902 - binary_accuracy: 0.5723 - val_loss: 0.6758 - val_binary_accuracy: 0.6098
Epoch 2/75
6/6 [==============================] - 12s 2s/step - loss: 0.6758 - binary_accuracy: 0.6084 - val_loss: 0.6689 - val_binary_accuracy: 0.6098
Epoch 3/75
6/6 [==============================] - 12s 2s/step - loss: 0.6679 - binary_accuracy: 0.6084 - val_loss: 0.6658 - val_binary_accuracy: 0.6098
Epoch 4/75
6/6 [==============================] - 12s 2s/step - loss: 0.6664 - binary_accuracy: 0.6084 - val_loss: 0.6608 - val_binary_accuracy: 0.6098
Epoch 5/75
6/6 [==============================] - 12s 2s/step - loss: 0.6634 - binary_accuracy: 0.6084 - val_loss: 0.6667 - val_binary_accuracy: 0.6098
Epoch 6/75
6/6 [==============================] - 12s 2s/step - loss: 0.6477 - binary_accuracy: 0.6265 - val_loss: 0.6523 - val_binary_accuracy: 0.6098
Epoch 7/75
6/6 [==============================] - 12s 2s/step - loss: 0.6375 - binary_ac

In [23]:
y_pred_1 = model_1.predict(X_test_1,verbose=0).round()
accuracy_1 = accuracy_score(y_test_1, y_pred_1)
print("Accuracy:", round(accuracy_1,2))

Accuracy: 0.67


In [24]:
test_labels_1 = model_1.predict(test_images_1,verbose=0).round()
print(test_labels_1.reshape(1,-1)[0].tolist())

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
